# Segmenting and Clustering Neighborhoods in Toronto - Assignment

#### Import the libraries to prepare the data

In [15]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

#### Create a soup objected from the wikipedia link containing the postal codes

In [16]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html_data = requests.get(url).text
soup = BeautifulSoup(html_data,"html5lib") 

#### Extract the data from the table and create a dictionary

In [17]:
table = soup.find("table")
df_content = []
for row in table.find_all("td") :
    cell = {}
    
    if row.span.text=='Not assigned':
        pass
    
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        df_content.append(cell)

#### Use the dictionary to create a data frame
##### Also, correct some of the names and join neighborhoods with the same postal code

In [42]:
df_column_names=['PostalCode', 'Borough', 'Neighborhood']
neighbourhoods_df=pd.DataFrame(data=df_content, columns=df_column_names)
neighbourhoods_df['Borough']=neighbourhoods_df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})


#### Show the shape of the resulting data frame

In [43]:
neighbourhoods_df.shape

(103, 3)

In [51]:
neighbourhoods_df[:11]

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


#### Get the coordinates and append them to the data frame

In [90]:
coordinates_url="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv"
coordinates_df=pd.read_csv(coordinates_url)

neighbourhoods_df['Latitude'] = neighbourhoods_df.PostalCode.map(coordinates_df.set_index('Postal Code')['Latitude'].to_dict())
neighbourhoods_df['Longitude'] = neighbourhoods_df.PostalCode.map(coordinates_df.set_index('Postal Code')['Longitude'].to_dict())

In [92]:
neighbourhoods_df.shape

(103, 5)

In [96]:
neighbourhoods_df[:11]

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
